In [ ]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.0 MB/s eta 0:00:0

In [ ]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [ ]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

In [ ]:
import json
import re
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate

class SleepCoachAgent:
    def __init__(self, api_key):
        self.llm = OpenAI(openai_api_key=api_key, model="gpt-4o-mini", max_tokens=4096)
        self.prompt = PromptTemplate(
    template = """
You are a smart, caring, and proactive Sleep Coach AI. Your role is to help users reflect on their sleep, understand behavioral patterns, and build sustainable habits for better rest and recovery.

Your job is to analyze the user's sleep-related input and provide a rich JSON response structured across multiple categories.

==============================
CONTEXT:
{context}

USER INPUT:
"{input}"
==============================

🔍 ANALYSIS GOALS:
1. Identify the user's sleep issue, need, or question
2. Extract important data (e.g., time, feelings, habits, metrics)
3. Provide a brief but insightful summary of the user's sleep pattern
4. Detect mood and emotional cues if present
5. Generate a personalized wind-down routine
6. Recommend optimal sleep and wake-up timing
7. Suggest behavior/environment improvements
8. Perform a basic sleep disorder screening
9. Propose follow-up actions and metrics to track
10. Tag context categories (e.g., stress, diet, tech use)

📦 RESPONSE FORMAT (STRICTLY VALID JSON):
Double quotes must be used.
Avoid any markdown formatting or extra text.
Response must end cleanly with the final `}}`.

RESPONSE FORMAT:
{{
  "sleepAssessment": {{
    "issue": "string",  // e.g. 'difficulty falling asleep'
    "confidence": number,  // 0.0 to 1.0
    "severity": "low" | "medium" | "high",
    "summary": "string",
    "sleepHistoryDetected": boolean,
    "userMood": "tired" | "refreshed" | "anxious" | "groggy" | "unknown",
    "sleepDurationTrend": "increasing" | "decreasing" | "stable" | "unknown"
  }},
  "routineRecommendation": {{
    "windDown": ["string"],  // calming activities
    "avoidBeforeBed": ["string"],  // caffeine, screens, etc.
    "optimalSleepTime": "string",  // e.g. "10:45 PM"
    "optimalWakeTime": "string",  // e.g. "6:30 AM"
    "reminders": ["string"]
  }},
  "tips": {{
    "immediateActions": ["string"],  // changes starting tonight
    "lifestyleChanges": ["string"],  // longer-term habits
    "environmentSuggestions": ["string"]  // bedroom, lighting, etc.
  }},
  "disorderCheck": {{
    "riskLevel": "low" | "medium" | "high",
    "symptoms": ["string"],
    "recommendations": ["string"],
    "complianceRisk": "low" | "medium" | "high"  // likelihood user may not follow
  }},
  "followUp": {{
    "checkInPeriod": "daily" | "weekly" | "custom",
    "trackMetrics": ["string"],  // e.g. "hours slept", "interruptions"
    "goals": ["string"]  // e.g. "reduce latency to < 20 minutes"
  }},
  "metadata": {{
    "wordCount": number,
    "timeOfDayMentioned": "morning" | "afternoon" | "evening" | "night" | "unknown",
    "contextTags": ["string"]  // e.g. ["stress", "caffeine", "screen use"]
  }}
}}
"""
,
    input_variables=["context", "input"]
)
        self.chain = self.prompt | self.llm
        self.collection = self._get_collection()

    def _get_collection(self):
        # You can customize this to use a DB or tracking log if needed
        return []

    def run(self, user_input, context=None):
      try:
          context_str = """
          - MindFuel's Sleep Coach helps users improve sleep hygiene and build better bedtime routines.
          - Users may be experiencing insomnia, irregular schedules, screen overuse, or stress.
          - The agent provides actionable suggestions and helps track progress over time.
          - All suggestions should be safe, natural, and non-prescriptive.
          """

          prompt_vars = {
              "context": context_str,
              "input": user_input
          }

          result = self.chain.invoke(prompt_vars)

          # print("-----RAW LLM OUTPUT-----")
          # print(result)
          # print("------------------------")

          try:
              # Try direct load first
              try:
                  return json.loads(result.strip())
              except json.JSONDecodeError:
                  # Wrap if the model returns a partial JSON (starting directly with keys)
                  fixed = result.strip()
                  if not fixed.strip().startswith("{"):
                      fixed = "{\n" + fixed + "\n}"
                  return json.loads(fixed)

          except Exception as parse_err:
              print("❌ JSON Parsing Error:", parse_err)
              return {"error": "Failed to parse JSON response."}

      except Exception as e:
          print("❌ Agent Run Error:", e)
          return {"error": str(e)}




In [ ]:
agent = SleepCoachAgent(api_key=key)
response = agent.run("I sleep 8 hours but still feel tired in the morning. I often use my phone before bed.")
print(json.dumps(response, indent=2))


{
  "sleepAssessment": {
    "issue": "persistent tiredness despite adequate sleep",
    "confidence": 0.8,
    "severity": "medium",
    "summary": "User sleeps for 8 hours but still feels tired in the morning, possibly due to phone use before bed.",
    "sleepHistoryDetected": false,
    "userMood": "tired",
    "sleepDurationTrend": "stable"
  },
  "routineRecommendation": {
    "windDown": [
      "reading a book",
      "meditation",
      "gentle stretching"
    ],
    "avoidBeforeBed": [
      "using phone",
      "caffeine",
      "heavy meals"
    ],
    "optimalSleepTime": "10:30 PM",
    "optimalWakeTime": "6:30 AM",
    "reminders": [
      "Set a screen time limit of 30 minutes before bed."
    ]
  },
  "tips": {
    "immediateActions": [
      "Stop using your phone 30 minutes before bed."
    ],
    "lifestyleChanges": [
      "Establish a consistent sleep schedule."
    ],
    "environmentSuggestions": [
      "Keep your bedroom dark and cool."
    ]
  },
  "disorderChe